<a href="https://colab.research.google.com/github/samohtwal/BPR/blob/main/BPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
import pandas as pd
from math import ceil
from tqdm import trange
from subprocess import call
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix

import copy

In [ ]:
names = ['user_id', 'item_id']
df = pd.read_csv('bpr_data/bpr_data.csv', sep = '\t', names = names)
print('data dimension: \n', df.shape)
df.head()

data dimension: 
 (5, 2)


,user_id,item_id
0,196,242
1,196,248
2,186,302
3,22,377
4,244,51


In [ ]:
def create_matrix(data_input, users_col, items_col, ratings_col):

    data = copy.deepcopy(data_input)

    data[ratings_col] = 1

    data_user_num_items = (data
                           .groupby('user_id')
                           .agg(**{'num_items': ('item_id', 'count')})
                           .reset_index())
    data = data.merge(data_user_num_items, on='user_id', how='inner')
    data = data[data['num_items'] > 1]

    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].astype('category')

    print('data: ' + data.to_string())

    ratings = csr_matrix(
                    (data[ratings_col],
                        (data[users_col].cat.codes, data[items_col].cat.codes)
                        ))
    ratings.eliminate_zeros()

    return ratings, data

In [ ]:
items_col = 'item_id'
users_col = 'user_id'
ratings_col = 'rating'
ratings, df_output = create_matrix(df, users_col, items_col, ratings_col)
print(ratings[0].todense())
print(ratings.indptr)
print(ratings.indices)
print(ratings.shape)

# df_output
# df2 = create_matrix(df, users_col, items_col, ratings_col, threshold)
# df2
# df = df[df[ratings_col] >= 2]
# df[ratings_col] = 1
# df

data:   user_id item_id rating  num_items
0     196     242      1          2
1     196     248      1          2
[[1 1]]
[0 2]
[0 1]
(1, 2)
